# 랭스미스 LangSmith

https://smith.langchain.com/

LangSmith는 OpenAI의 언어 모델(ChatGPT 등)을 활용하여 맞춤형 언어 애플리케이션을 개발하고 디버깅하기 위한 도구 및 플랫폼이다.

개발자가 자연어 처리 기반의 애플리케이션을 효과적으로 설계하고 관리할 수 있도록 다양한 기능을 제공한다.

1. **트레이싱 및 디버깅(Tracing & Debugging)**  
   - 생성된 응답을 추적하고, 언어 모델의 동작을 분석하여 결과를 최적화할 수 있다.
   - 디버깅 도구를 통해 애플리케이션의 흐름에서 문제를 쉽게 식별하고 수정 가능.

2. **모델 조정(Customization)**  
   - 기본 모델 외에도 프롬프트 설계(prompt engineering)와 파인튜닝(fine-tuning)을 활용하여 애플리케이션에 적합한 응답을 생성할 수 있다.

3. **평가 및 테스트(Evaluation & Testing)**  
   - 다양한 입력과 환경에서 모델의 성능을 테스트하고, 모델 결과를 체계적으로 비교 가능.
   - 지속적으로 성능을 측정해 품질을 유지.

4. **워크플로우 통합(Workflow Integration)**  
   - API 및 SDK를 제공하여 LangChain과 같은 프레임워크와 손쉽게 통합 가능.
   - 빠르고 직관적인 프로토타입 제작 지원.

5. **버전 관리(Versioning)**  
   - 모델 및 프롬프트 변경 사항을 체계적으로 기록하여 실험의 재현성과 관리 용이.


> LangChain으로 빌드하는 많은 애플리케이션은 LLM 호출을 여러 번 호출하는 아키텍처를 가질 수 있다.
>
> 이러한 애플리케이션이 점점 더 복잡해짐에 따라 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지를 검사하는 것이 중요해진다.
>
> 랭스미스(LangSmith)는 이러한 단계를 모니터링 할 수 있게 해주기 때문에 LLM 애플리케이션을 개발 할 때 랭스미스를 사용하는 것을 권장한다.

In [2]:
# -U : 업그레이드 (설치되있으면 최신버전으로, 아니면 최신버전 설치)
%pip install -U langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                   # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")  # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

In [5]:
from langchain.agents import create_agent

# 도시명을 받아 "날씨"를 포함한 문자열을 반환하는 함수
def get_weather(city: str) -> str:
    """Get weather for a given city"""
    return f"It's always sunny in {city}"

# 에이전트 생성
agent = create_agent(
    model = 'openai:gpt-5-mini',  # 사용할 모델
    tools = [get_weather],        # 에이전트가 사용할 툴 목록
    system_prompt = "You are helpful assistant"  # 시스템 프롬프트
)

# 에이전트 실행
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What is the weather in San Francisco?"}]}
)

response  # 응답 전체

{'messages': [HumanMessage(content='What is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='66b65972-854a-45e0-8e1a-85cadeaa21dd'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 141, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5PdocGODz3QuwGJib28W0ORyzUjL', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c271a-8576-7a71-8d39-156e01f0f8e9-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_rgodsE6jhykYPfG4kplyvceD', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 141, 

In [11]:
response['messages'][0]  # HumanMessage

HumanMessage(content='What is the weather in San Francisco?', additional_kwargs={}, response_metadata={}, id='66b65972-854a-45e0-8e1a-85cadeaa21dd')

In [12]:
response['messages'][-1]   # AIMessage

AIMessage(content='The weather tool reports: "It\'s always sunny in San Francisco."\n\nWould you like more specific information (current temperature, hourly or 7‑day forecast, wind, or radar)?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 237, 'prompt_tokens': 176, 'total_tokens': 413, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D5PdqCiLJWsbiBd619tPD70nHddHj', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c271a-8e1f-77a3-b9c9-99369225cf0d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 237, 'total_tokens': 413, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_det

In [6]:
response['messages'][0].content  # HumanMessage 질문

'What is the weather in San Francisco?'

In [8]:
response['messages'][-1].content  # AIMessage 답변

'The weather tool reports: "It\'s always sunny in San Francisco."\n\nWould you like more specific information (current temperature, hourly or 7‑day forecast, wind, or radar)?'